In [1]:
from time import sleep
from IPython.display import Image
import folium
import geehydro
from datetime import datetime as dt
import numpy as np
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/2QHOEiQ9AL6xQTZ3XyrE9wGsTk4flsWzlwPdMzCHZ7Q4LB8eX5ELCjg

Successfully saved authorization token.


In [2]:
def maskS2clouds(image):
    """
     Function to mask clouds using the Sentinel-2 QA band
     @param {ee.Image} image Sentinel-2 image
     @return {ee.Image} cloud masked Sentinel-2 image
     """
    qa = image.select('QA60');

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = (1 << 10)
    cirrusBitMask = (1 << 11)

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) and (qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000)

## Image collections

**Landsat-8**

**Sentinel-2 MSI: MultiSpectral Instrument, Level-2A**

#### export tiles of the toyohashi area
Coordinates:
34.65 - 34.85
137.3 - 137.5

Resolution:
0.025 x 0.025

In [36]:
def grid_pairs(min_x, max_x, min_y, max_y, resolution):
    # transform to origin
    nx = np.round((max_x-min_x)/resolution).astype(np.int)+1
    ny = np.round((max_y-min_y)/resolution).astype(np.int)+1
    x = np.linspace(min_x, max_x, nx)
    y = np.linspace(min_y, max_y, ny)
    xv, yv = np.meshgrid(x, y)
    grid_points = list(zip(yv.flatten(), xv.flatten()))
    # print(grid_points)
    skips = int((max_x-min_x)/resolution)+2
    count = 0
    pairs_coords = []
    for i, point in enumerate(grid_points[:-skips]):
        if point[1]==max_x:
            continue
        count+=1
        pairs_coords.append([point[0], point[1], grid_points[i+skips][0], grid_points[i+skips][1]])
    return pairs_coords
# grid_pairs(34.675,34.725,137.35,137.4,0.025)
# grid_pairs(34.675,34.725,137.4,137.45,0.025)

In [35]:
# coords = grid_pairs(34.65,34.85,137.3,137.5,0.025) # all tiles
# coords = grid_pairs(34.675,34.725,137.35,137.4,0.025) # greenhouse samples train
coords = grid_pairs(34.675,34.725,137.4,137.45,0.025) # greenhouse samples eval


# another export format
landsat = ee.ImageCollection("COPERNICUS/S2_SR")\
            .filterDate('2018-01-01', '2018-12-31')\
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
            .map(maskS2clouds).select(['B4', 'B3', 'B2'])

# export Area of Interests (AOI)
count = 0
for coord in coords:
    name = "{:.4f}_{:.4f}".format(coord[0], coord[1])
    count += 1
    print(f"{count}: {name}")
    export_AOI = ee.Geometry.Rectangle(coord)

    # filter area
    landsat_AOI = landsat.filterBounds(export_AOI)

    least_cloudy = ee.Image(landsat_AOI.sort('LAND_COVER').first())
    
    task_config = {
        'scale': 1,
        'region': export_AOI,
        'folder':'toyohashi_sample_greenhouses_sentinel_eval',
        'crs': 'EPSG:4326'
        }
    task = ee.batch.Export.image(least_cloudy, name, task_config)
    task.start()
    sleep(1)
print("DONE!!!")

1: 137.4000_34.6750
2: 137.4000_34.7000
3: 137.4250_34.6750
4: 137.4250_34.7000
DONE!!!
